In [ ]:
import ipywidgets.widgets as widgets
from time import sleep
import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.camera import get_images_and_cameras
from ballsbot.servos import get_controls
from ballsbot.utils import keep_rps, run_as_thread
from ballsbot.lidar import Lidar
from ballsbot.session import start_rospy_session
from ballsbot.manipulator import Manipulator
from ballsbot.imu import IMU_Threaded
from ballsbot.drawing import ManipulatorPoseDrawing

In [ ]:
start_rospy_session('a_car')

In [ ]:
controller = widgets.Controller(index=0)  # http://html5gamepad.com
display(controller)

images = []
for image, camera in get_images_and_cameras():
    images.append(image)
hbox = widgets.HBox(images)
display(hbox)

lidar_image = widgets.Image(format='jpeg', width=600, height=600)
painter = ManipulatorPoseDrawing(widgets)
display(widgets.HBox([
    lidar_image,
    widgets.VBox([painter.get_images(), widgets.Image(format='jpeg', width=400, height=200)])
]))

In [ ]:
manipulator = Manipulator(controller)

In [ ]:
lidar = Lidar()

# lidar.calibrate()
# print(lidar.calibration)

In [ ]:
def lidar_run():
    lidar.show_lidar_cloud(lidar_image)

run_as_thread(lidar_run)

In [ ]:
imu = IMU_Threaded()

In [ ]:
car_controls = get_controls()

def update_steering(value):
    car_controls['steering'].run(value['new'])

def update_throttle(value):
    car_controls['throttle'].run(-value['new'])

def link_controls():
    ts = None
    while True:
        ts = keep_rps(ts, fps=1)

        if len(controller.axes):
            controller.axes[0].observe(update_steering, 'value')
            controller.axes[1].observe(update_throttle, 'value')
            break

run_as_thread(link_controls)

In [ ]:
def update_manipulator_images():
    ts = None
    while True:
        ts = keep_rps(ts, fps=1)
        pose = manipulator.get_manipulator_pose()
        painter.update_images(pose, override_crop_half_size=550.)

run_as_thread(update_manipulator_images)

In [ ]:
# import json
# json.dumps(lidar._get_lidar_points(8.))

In [ ]:
# for cam in cameras:
#     cam.stop()

In [ ]:
# from ballsbot.utils import join_all_theads
# join_all_theads()